In [3]:
### pip install twilio

In [3]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [1]:
query = 'Montevideo'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

NameError: name 'API_KEY_WAPI' is not defined

In [5]:
response = requests.get(url_clima).json()

In [1]:
## response

In [7]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [8]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [9]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [10]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1735182000,
 'time': '2024-12-26 00:00',
 'temp_c': 18.6,
 'temp_f': 65.5,
 'is_day': 0,
 'condition': {'text': 'Patchy rain nearby',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 16.1,
 'wind_kph': 25.9,
 'wind_degree': 178,
 'wind_dir': 'S',
 'pressure_mb': 1012.0,
 'pressure_in': 29.88,
 'precip_mm': 0.14,
 'precip_in': 0.01,
 'snow_cm': 0.0,
 'humidity': 78,
 'cloud': 100,
 'feelslike_c': 18.6,
 'feelslike_f': 65.5,
 'windchill_c': 18.6,
 'windchill_f': 65.5,
 'heatindex_c': 18.6,
 'heatindex_f': 65.5,
 'dewpoint_c': 14.7,
 'dewpoint_f': 58.4,
 'will_it_rain': 1,
 'chance_of_rain': 100,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 22.5,
 'gust_kph': 36.3,
 'uv': 0}

In [23]:
# Obtengo fecha, hora y demás datos interesantes

fecha = response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]
hora = response['forecast']['forecastday'][0]['hour'][0]['time'].split()[1].split(':')
condicion = response['forecast']['forecastday'][0]['hour'][0]['condition']
tempC = response['forecast']['forecastday'][0]['hour'][0]['temp_c']
willItRain = response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']
channceRain = response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

In [24]:
print(fecha)
print(hora)
print(condicion)

print(tempC)
print(willItRain)
print(channceRain)

2024-12-26
['00', '00']
{'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png', 'code': 1063}
18.6
1
100


In [27]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [28]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))

100%|████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 5993.29it/s]


In [29]:
datos[0]

('2024-12-26', 0, 'Patchy rain nearby', 18.6, 1, 100)

In [31]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-12-26,0,Patchy rain nearby,18.6,1,100
1,2024-12-26,1,Light rain shower,18.4,1,100
2,2024-12-26,2,Light rain shower,18.4,1,100
3,2024-12-26,3,Patchy rain nearby,18.5,1,100
4,2024-12-26,4,Light rain shower,18.4,1,100
5,2024-12-26,5,Light rain shower,18.5,1,100
6,2024-12-26,6,Patchy rain nearby,18.6,1,100
7,2024-12-26,7,Patchy rain nearby,19.0,1,86
8,2024-12-26,8,Patchy rain nearby,19.7,1,85
9,2024-12-26,9,Patchy rain nearby,20.6,1,77


In [33]:
# df[df['Lluvia']==1]

df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [35]:
df_rain

,Condicion
Hora,
7,Patchy rain nearby
8,Patchy rain nearby
9,Patchy rain nearby
10,Patchy rain nearby
11,Patchy rain nearby
13,Patchy rain nearby
14,Patchy rain nearby
15,Patchy rain nearby
16,Patchy rain nearby


In [37]:
strMensaje = '\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

In [39]:
print(strMensaje)


Hola! 


 El pronostico del tiempo hoy 2024-12-26 en Montevideo es : 


                Condicion
Hora                    
7     Patchy rain nearby
8     Patchy rain nearby
9     Patchy rain nearby
10    Patchy rain nearby
11    Patchy rain nearby
13    Patchy rain nearby
14    Patchy rain nearby
15    Patchy rain nearby
16    Patchy rain nearby
18    Patchy rain nearby


In [40]:
PHONE_NUMBER

'+1 762 255 9233'

In [42]:
# MENSAJE CON TWILIO

time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMba28b65c71a64925e21e0911fd8a04e4


In [48]:
from twilio.rest import Client

account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:+',
  body='CHUPALA',
  to='whatsapp:+'
)

print(message.sid)

SM013b88eceb09949daf03917a79ae233c
